# Module 5 Homework - Let's Get Lunch

Suppose we are looking for a good place for lunch on our next trip to La Crosse. Let's collect some data on nearby restaurants.

## <font color="red"> Problem 1 </font>

Goto yelp.com and perform a search with the following parameters

* **Find** Restaurants
* **Near** La Crosse, WI

**Tasks**
1. Copy the resulting web address below and determine the how the     specified search terms related to the resulting address
2. Use requests and Beautiful Soap to download the content of the front page.

In [ ]:
# Import modules here
import requests
from bs4 import BeautifulSoup
from composablesoup import find, find_all, get_text, has_attr
from composable.sequence import slice, head
from composable.strict import map, filter
from composable.string import replace, split
from composable import from_toolz as tlz
from composable import pipeable
from composable import pipeable
from composable.strict import map, filter
from composablesoup import find, find_all, get_text, has_attr
from composablesoup.soup import find_parent, parents, children, find_previous_sibling, find_previous_siblings, find_next_sibling, find_next_siblings, find_previous_sibling
from composable.sequence import to_list, head
from composable.string import strip
from composable import from_toolz as tlz

In [ ]:
# Get and process the Yelp search
s = requests.Session()
r = s.get('https://www.yelp.com/search?find_desc=&find_loc=La%20Crosse%2C%20WI')
yelp = BeautifulSoup(r.content, "html.parser")

## <font color="red"> Problem 2 </font>

We want to grab the restaurant's name.

1. Use Inspect Element to determine the tags/classes for each of the elements listed above.  
2. Note that all the business information is contained in a`div` that contains a class similar to  `<div class=" ... businessName__09f24__3Wql2 ...">`.  You will need to use a regular expression to match the `businessName` in the class (see lecture 5.3).
3. Write expressions/functions to pull out the name of each restaurant.  
    * Note: The business name is in an unnamed tag, you will need to navigate to the using searches and/or relationship
 
**Confirm that there is an extra restaurant in the list (e.g. 11-12 instead of 10). This is due to an advertisement/sponsered links, which we want to ignore.**

In [ ]:
import re

(yelp
    >> find_all('a', attrs={'class':"link__09f24__1kwXV link-color--inherit__09f24__3PYlA link-size--inherit__09f24__2Uj95"})
    >> map(get_text)
)



In [ ]:
get_all_business = lambda soup: (soup    
                    >> find_all('a', attrs={'class':"link__09f24__1kwXV link-color--inherit__09f24__3PYlA link-size--inherit__09f24__2Uj95", 
    'href':not_ad})
                    >> map(get_text))
get_all_business(yelp)

## <font color="red"> Problem 3 </font>

Since we picked up extra information, we will need to be clever about identifying the information block for each restaurant.  Note that all of the actual search results (but not sponsered links) start with the ranking (e.g. `11.`).  Use the following steps to get a list that contains the information for each restaurant other than the adds.

1. Start by finding the ranking of the restaurant (1., 2., etc.). **Hint:** You will need to use regular expression to match the text of the tag (see lecture 5.3).
2. Now search for a parent of the above tags that surrounds all of the restaurant information.  You will want to use the `find_parent` method on each of the tags from **1.**.  **Hint:** Look through each of the `div` tags that contain the ranking, looking for a meaningful tag name to match with a regular expression.

The resulting list will be the starting point for gathering all of the information.

In [ ]:
import re

not_ad = re.compile('/biz/')

(yelp
    >> find_all('a', attrs={'class':"link__09f24__1kwXV link-color--inherit__09f24__3PYlA link-size--inherit__09f24__2Uj95", 
    'href':not_ad})
    >> map(get_text)
)

In [ ]:
get_only_business = lambda soup: (soup     
                        >> find_all('a', attrs={'class':"link__09f24__1kwXV link-color--inherit__09f24__3PYlA link-size--inherit__09f24__2Uj95", 
    'href':not_ad})
                        >> map(get_text)
)
get_only_business(yelp)

## <font color="red"> Problem 4 </font>

Write expressions/functions to gather each of the following pieces of information for each of the restaurants.

#### Restaurant Name

In [ ]:
get_only_business = lambda soup: (soup     
                        >> find_all('a', attrs={'class':"link__09f24__1kwXV link-color--inherit__09f24__3PYlA link-size--inherit__09f24__2Uj95", 
    'href':not_ad})
                        >> map(get_text)
)
get_only_business(yelp)

#### Rating

In [ ]:
rating = re.compile('\d[\.\d]? star rating')
name = re.compile('.+')


(yelp 
    >> find_all('a', attrs={'href':not_ad, 'name':name})
    >> map(find_parent)
    >> map(find_parent)
    >> map(find_parent)
    >> map(find_parent)
    >> map(find_parent)
    >> map(find_next_sibling)
    >> map(find('div', attrs={'aria-label':rating, 'role':'img'}))
    >> map(tlz.get('aria-label'))
)

In [ ]:
get_rating = lambda soup: (soup     
    >> find_all('a', attrs={'href':not_ad, 'name':name})
    >> map(find_parent)
    >> map(find_parent)
    >> map(find_parent)
    >> map(find_parent)
    >> map(find_parent)
    >> map(find_next_sibling)
    >> map(find('div', attrs={'aria-label':rating, 'role':'img'}))
    >> map(tlz.get('aria-label'))
)
get_rating(yelp)

#### Address

In [ ]:
(yelp
    >> find_all('a', attrs={'href':not_ad, 'name':name})
    >> map(find_parent)
    >> map(find_parent)
    >> map(find_parent)
    >> map(find_parent)
    >> map(find_parent)
    >> map(find_parent)
    >> map(find_parent)
    >> map(find_next_sibling)
    >> map(find('span'))
    >> map(get_text)
)

In [ ]:
get_address = lambda soup: (soup
    >> find_all('a', attrs={'href':not_ad, 'name':name})
    >> map(find_parent)
    >> map(find_parent)
    >> map(find_parent)
    >> map(find_parent)
    >> map(find_parent)
    >> map(find_parent)
    >> map(find_parent)
    >> map(find_next_sibling)
    >> map(find('span'))
    >> map(get_text)
)
get_address(yelp)

#### Review Count

In [ ]:
flatten_pipe = pipeable(lambda s: [item for sublist in s for item in sublist])

(yelp
    >> find_all('a', attrs={'href':not_ad, 'name':name})
    >> map(find_parent)
    >> map(find_parent)
    >> map(find_parent)
    >> map(find_parent)
    >> map(find_parent)
    >> map(find_next_sibling)
    >> map(map(get_text))
    >> flatten_pipe
)

In [ ]:
flatten_pipe = pipeable(lambda s: [item for sublist in s for item in sublist])
get_review_count = lambda soup: (soup
    >> find_all('a', attrs={'href':not_ad, 'name':name})
    >> map(find_parent)
    >> map(find_parent)
    >> map(find_parent)
    >> map(find_parent)
    >> map(find_parent)
    >> map(find_next_sibling)
    >> map(map(get_text))
    >> flatten_pipe
)

get_review_count(yelp)

#### Category

**Examples:** `['American (New)', 'Seafood', 'Steakhouses']` becomes `'American (New);Seafood;Steakhouses'`

In [ ]:
(yelp
    >> find_all('a', attrs={'href':not_ad, 'name':name})
    >> map(find_parent)
    >> map(find_parent)
    >> map(find_parent)
    >> map(find_parent)
    >> map(find_parent)
    >> map(find_next_sibling)
    >> map(find_next_sibling)
    >> map(map(get_text))
    >> map(map(replace('$', '')))
    >> map(map(replace(', ', ';')))
    >> flatten_pipe
)

In [ ]:
get_category = lambda soup: (soup
    >> find_all('a', attrs={'href':not_ad, 'name':name})
    >> map(find_parent)
    >> map(find_parent)
    >> map(find_parent)
    >> map(find_parent)
    >> map(find_parent)
    >> map(find_next_sibling)
    >> map(find_next_sibling)
    >> map(map(get_text))
    >> map(map(replace('$', '')))
    >> map(map(replace(', ', ';')))
    >> flatten_pipe
)

## <font color="red">  Problem 4 </font>

Package all of the expressions in a function that takes a url as input and returns the table of information.  Use a `def` statement and put the above helper functions in the body of the main function.  Test this function on the front page of the search.

In [ ]:
def get_yelp_information(url):
    """This function returns various components from Yelp for La Crosse restaurants
    Args:
        soup: Takes a Yelp page
    Returns:
        Returns a table with name, address, rating, count of ratings, and category
    """

    s = requests.Session()
    r = s.get('https://www.yelp.com/search?find_desc=&find_loc=La%20Crosse%2C%20WI')
    soup = BeautifulSoup(r.content, "html.parser")

    get_only_business = lambda soup: (soup     
        >> find_all('a', attrs={'class':"link__09f24__1kwXV link-color--inherit__09f24__3PYlA link-size--inherit__09f24__2Uj95", 
'href':not_ad})
        >> map(get_text)
    )

    get_rating = lambda soup: (soup     
        >> find_all('a', attrs={'href':not_ad, 'name':name})
        >> map(find_parent)
        >> map(find_parent)
        >> map(find_parent)
        >> map(find_parent)
        >> map(find_parent)
        >> map(find_next_sibling)
        >> map(find('div', attrs={'aria-label':rating, 'role':'img'}))
        >> map(tlz.get('aria-label')))

    get_address = lambda soup: (soup
        >> find_all('a', attrs={'href':not_ad, 'name':name})
        >> map(find_parent)
        >> map(find_parent)
        >> map(find_parent)
        >> map(find_parent)
        >> map(find_parent)
        >> map(find_parent)
        >> map(find_parent)
        >> map(find_next_sibling)
        >> map(find('span'))
        >> map(get_text))

    flatten_pipe = pipeable(lambda s: [item for sublist in s for item in sublist])
    get_review_count = lambda soup: (soup
        >> find_all('a', attrs={'href':not_ad, 'name':name})
        >> map(find_parent)
        >> map(find_parent)
        >> map(find_parent)
        >> map(find_parent)
        >> map(find_parent)
        >> map(find_next_sibling)
        >> map(map(get_text))
        >> flatten_pipe)

    get_category = lambda soup: (soup
        >> find_all('a', attrs={'href':not_ad, 'name':name})
        >> map(find_parent)
        >> map(find_parent)
        >> map(find_parent)
        >> map(find_parent)
        >> map(find_parent)
        >> map(find_next_sibling)
        >> map(find_next_sibling)
        >> map(map(get_text))
        >> map(map(replace('$', '')))
        >> map(map(replace(', ', ';')))
        >> flatten_pipe)  

    name = get_only_business(soup)
    address = get_address(soup)
    ratings = get_rating(soup)
    rating_count = get_review_count(soup)
    category = get_category(soup)

    list_of_tuples = zip(name, address, ratings, rating_count, category)

    output = [', '.join(x) for x in list_of_tuples]

    return(output)

get_yelp_information(yelp)

## <font color="red">  Problem 5 </font>

Now perform a linear search to grab all of the information on restaurants in La Crosse.  You will need to

1. Inspect the url for the first, second, etc. pages to determine the pattern.
2. Create a list of urls for all search results.
3. Get the info from all pages and use your function from the last problem to parse the data into a list.
4. Write the results to a csv file. **Hint:** Use `'a'` (append) instead of `'w'` write on all after the first/

In [207]:
base_url = 'https://www.yelp.com/search?find_desc=&find_loc=La%20Crosse%2C%20WI&start='

#Last page = https://www.yelp.com/search?find_desc=&find_loc=La%20Crosse%2C%20WI&start=230

pages = [base_url + str(x) for x in range(240) if x % 10 == 0]
len(pages), pages




(24,
 ['https://www.yelp.com/search?find_desc=&find_loc=La%20Crosse%2C%20WI&start=0',
  'https://www.yelp.com/search?find_desc=&find_loc=La%20Crosse%2C%20WI&start=10',
  'https://www.yelp.com/search?find_desc=&find_loc=La%20Crosse%2C%20WI&start=20',
  'https://www.yelp.com/search?find_desc=&find_loc=La%20Crosse%2C%20WI&start=30',
  'https://www.yelp.com/search?find_desc=&find_loc=La%20Crosse%2C%20WI&start=40',
  'https://www.yelp.com/search?find_desc=&find_loc=La%20Crosse%2C%20WI&start=50',
  'https://www.yelp.com/search?find_desc=&find_loc=La%20Crosse%2C%20WI&start=60',
  'https://www.yelp.com/search?find_desc=&find_loc=La%20Crosse%2C%20WI&start=70',
  'https://www.yelp.com/search?find_desc=&find_loc=La%20Crosse%2C%20WI&start=80',
  'https://www.yelp.com/search?find_desc=&find_loc=La%20Crosse%2C%20WI&start=90',
  'https://www.yelp.com/search?find_desc=&find_loc=La%20Crosse%2C%20WI&start=100',
  'https://www.yelp.com/search?find_desc=&find_loc=La%20Crosse%2C%20WI&start=110',
  'https:/

In [ ]:
for pages in pages:
    info = get_yelp_information(pages)
    

In [228]:
remaining_pages = pages[1:]

with open('la_crosse_yelp.csv', 'w') as file:
    page_info = get_yelp_information(pages[0])
    rows = '\n'.join(page_info) + '\n'
    file.write(rows)

    print('Completed the first url')

for remaining_pages in remaining_pages:
    print('Starting for url: ' + remaining_pages)
    with open('la_crosse_yelp.csv', 'a') as file:
        page_info = get_yelp_information(remaining_pages)
        rows = '\n'.join(page_info) + '\n'
        file.writelines(rows)
print('The script is complete')

Completed the first url
Starting for url: https://www.yelp.com/search?find_desc=&find_loc=La%20Crosse%2C%20WI&start=10
Starting for url: https://www.yelp.com/search?find_desc=&find_loc=La%20Crosse%2C%20WI&start=20
Starting for url: https://www.yelp.com/search?find_desc=&find_loc=La%20Crosse%2C%20WI&start=30
Starting for url: https://www.yelp.com/search?find_desc=&find_loc=La%20Crosse%2C%20WI&start=40
Starting for url: https://www.yelp.com/search?find_desc=&find_loc=La%20Crosse%2C%20WI&start=50
Starting for url: https://www.yelp.com/search?find_desc=&find_loc=La%20Crosse%2C%20WI&start=60
Starting for url: https://www.yelp.com/search?find_desc=&find_loc=La%20Crosse%2C%20WI&start=70
Starting for url: https://www.yelp.com/search?find_desc=&find_loc=La%20Crosse%2C%20WI&start=80
Starting for url: https://www.yelp.com/search?find_desc=&find_loc=La%20Crosse%2C%20WI&start=90
Starting for url: https://www.yelp.com/search?find_desc=&find_loc=La%20Crosse%2C%20WI&start=100
Starting for url: https:/

## <font color="red">  Bonus Problem </font>

See if you can also grab the latitude and longitude of each result.